# AI Camp Holiday Mission - Jupyter 데모

이 노트북은 AI Camp Holiday Mission 프로젝트의 주요 기능들을 시연합니다.

## 주요 기능
1. 야놀자 리뷰 크롤링
2. 리뷰 데이터 전처리
3. OpenAI를 활용한 리뷰 요약
4. Gradio 웹 인터페이스


## 1. 필요한 라이브러리 import


In [ ]:
import json
import os
import datetime
from dateutil import parser
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI
import gradio as gr

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


## 2. 환경 변수 설정

OpenAI API 키를 설정해주세요.


In [ ]:
# OpenAI API 키 설정 (실제 키로 교체하세요)
os.environ['OPENAI_API_KEY'] = 'your-openai-api-key-here'

# 또는 직접 설정
# OPENAI_API_KEY = 'your-openai-api-key-here'


## 3. 리뷰 데이터 로드 및 분석


In [ ]:
def load_review_data(file_path):
    """리뷰 데이터를 로드하는 함수"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return []

# 리뷰 데이터 로드 (예시)
review_files = {
    '인사동': './res/reviews.json',
    '판교': './res/ninetree_pangyo.json',
    '용산': './res/ninetree_yongsan.json'
}

print("사용 가능한 리뷰 파일:")
for name, path in review_files.items():
    if os.path.exists(path):
        print(f"✓ {name}: {path}")
    else:
        print(f"✗ {name}: {path} (파일 없음)")


## 4. 리뷰 데이터 전처리 및 시각화


In [ ]:
def analyze_reviews(review_data):
    """리뷰 데이터를 분석하는 함수"""
    if not review_data:
        return None
    
    df = pd.DataFrame(review_data)
    
    # 날짜 변환
    current_date = datetime.datetime.now()
    df['parsed_date'] = df['date'].apply(lambda x: parser.parse(x) if x else current_date)
    
    # 최근 6개월 필터링
    six_months_ago = current_date - datetime.timedelta(days=6*30)
    df_recent = df[df['parsed_date'] >= six_months_ago]
    
    # 긴 리뷰만 필터링 (30자 이상)
    df_filtered = df_recent[df_recent['review'].str.len() >= 30]
    
    return df_filtered

# 첫 번째 사용 가능한 파일로 분석
for name, path in review_files.items():
    if os.path.exists(path):
        print(f"\n{name} 리뷰 데이터 분석:")
        reviews = load_review_data(path)
        df = analyze_reviews(reviews)
        
        if df is not None and not df.empty:
            print(f"총 리뷰 수: {len(df)}")
            print(f"평균 별점: {df['stars'].mean():.2f}")
            print(f"별점 분포:")
            print(df['stars'].value_counts().sort_index())
            
            # 별점 분포 시각화
            plt.figure(figsize=(10, 6))
            
            plt.subplot(1, 2, 1)
            df['stars'].value_counts().sort_index().plot(kind='bar')
            plt.title(f'{name} - 별점 분포')
            plt.xlabel('별점')
            plt.ylabel('리뷰 수')
            
            plt.subplot(1, 2, 2)
            df['review'].str.len().hist(bins=20)
            plt.title(f'{name} - 리뷰 길이 분포')
            plt.xlabel('리뷰 길이 (문자 수)')
            plt.ylabel('빈도')
            
            plt.tight_layout()
            plt.show()
            
            # 최근 리뷰 몇 개 출력
            print("\n최근 리뷰 샘플:")
            for idx, row in df.head(3).iterrows():
                print(f"별점: {row['stars']}, 날짜: {row['date']}")
                print(f"리뷰: {row['review'][:100]}...")
                print("-" * 50)
        break


## 5. 프로젝트 구조 및 사용법


In [ ]:
print("AI Camp Holiday Mission 프로젝트 구조:")
print("=" * 50)
print("📁 프로젝트 루트")
print("├── 📄 crawler.py          # 야놀자 리뷰 크롤링")
print("├── 📄 demo.py             # Gradio 웹 인터페이스")
print("├── 📄 sample_demo.py      # 샘플 데모")
print("├── 📄 requirements.txt    # Python 패키지 목록")
print("├── 📄 setup_venv.bat      # 가상환경 설정 스크립트")
print("├── 📄 ai_camp_demo.ipynb  # 이 Jupyter 노트북")
print("├── 📁 venv/               # Python 가상환경")
print("└── 📁 res/                # 리뷰 데이터 저장소")
print("")
print("사용법:")
print("1. 가상환경 설정: setup_venv.bat 실행")
print("2. 가상환경 활성화: venv\\Scripts\\activate.bat")
print("3. Jupyter 실행: jupyter lab")
print("4. Gradio 데모: python demo.py")
print("5. 리뷰 크롤링: python crawler.py [숙소명] [URL]")
